In [ ]:
#PRUEBA DE REQUEST A WIKIPEDIA API

import requests

# URL de la API de Wikipedia (para obtener datos en formato JSON)
url = "https://es.wikipedia.org/w/api.php"

# Parámetros de la solicitud para obtener el resumen de una página
params = {
    "action": "query",
    "prop": "extracts",
    "exintro": True,
    "titles": "Albert Einstein",
    "format": "json"
}

# Hacer la solicitud GET a la API de Wikipedia
response = requests.get(url, params=params)

# Comprobar si la respuesta fue exitosa
if response.status_code == 200:
    # Obtener los datos en formato JSON
    data = response.json()
    pages = data["query"]["pages"]
    for page_id, page_data in pages.items():
        print(f"Título: {page_data['title']}")
        print(f"Extracto: {page_data['extract']}")
else:
    print(f"Error al conectarse a Wikipedia: {response.status_code}")


In [2]:
pip install --upgrade requests


  Attempting uninstall: requests
    Found existing installation: requests 2.7.0
    Uninstalling requests-2.7.0:
      Successfully uninstalled requests-2.7.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests

# URL para la API de Instant Answers de DuckDuckGo
url = "https://api.duckduckgo.com/"

# Parámetros de la solicitud
params = {
    "q": "¿Cuál es el significado de la vida?",  # La pregunta que deseas hacer
    "format": "json",  # Formato de respuesta
    "RelatedTopics":True, 
    #"no_html": 1,  # Opcional, para evitar HTML en la respuesta
    #"skip_disambig": 1  # Opcional, para evitar respuestas de desambiguación
}

# Enviar la solicitud GET
response = requests.get(url, params=params)

# Verificar la respuesta
if response.status_code == 200:
    print("Respuesta de DuckDuckGo:", response.json())
else:
    print(f"Error: {response.status_code}")


ImportError: cannot import name 'Mapping' from 'collections' (c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\collections\__init__.py)

In [ ]:
import requests

url = "https://openlibrary.org/search.json"
params = {
    "title": "The Meaning of Life"  # Cambia al título o tema deseado
}

response = requests.get(url, params=params)
if response.status_code == 200:
    books = response.json()["docs"]
    for book in books[:5]:  # Muestra los primeros 5 resultados
        print("Título:", book.get("title"), "| Autor:", book.get("author_name"))
else:
    print("Error:", response.status_code)


In [1]:
#PRUEBA DE SOLICITUD API A OLLAMA MODEL EN DOCKER

import requests

# URL correcta para interactuar con el API de Ollama
url = "http://middleearthar.ddns.net:11434/api/generate"

# Datos de la solicitud
data = {
    "model": "llama3.1",  # Cambia al modelo que deseas usar
    "prompt": "Responde a la siguiente pregunta en formato JSON: creame un post de instagram sobre kubernetes",
    #"format": "json",  # El formato de la respuesta
    "stream": False  # Opcional, pero si no quieres respuestas por streaming
}
# Enviar la solicitud POST
response = requests.post(url, json=data)

# Verificar la respuesta
if response.status_code == 200:
    print("Respuesta de Ollama:", response.json())
else:
    print(f"Error: {response.status_code}")


Respuesta de Ollama: {'model': 'llama3.1', 'created_at': '2024-11-09T13:22:04.608028488Z', 'response': '```json\n{\n  "texto": "¡Descubre el poder de Kubernetes! Con esta tecnología de containerización, puedes gestionar y escalarte aplicaciones con facilidad. ¿Quieres saber más? Mírate la conferencia que te compartimos en la carpeta \'Resursos\' ",\n  "imagenes": [\n    {\n      "url": "https://picsum.photos/200",\n      "descripcion": "Una imagen de un contenedor ejecutándose"\n    },\n    {\n      "url": "https://picsum.photos/201",\n      "descripcion": "Una imagen del diagrama de Kubernetes"\n    }\n  ],\n  "hashtags": [\n    "#Kubernetes",\n    "#Containerización",\n    "#DesarrolloDeSoftware",\n    "#IngenieríaDelSoftware"\n  ]\n}\n```', 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 1079, 76149, 264, 1208, 56001, 80440, 665, 55956, 4823, 25, 1922, 373, 653, 1772, 409, 52104, 15482, 597, 30927, 128009, 128006, 78191, 128007, 271, 74694, 2285, 198, 517,

In [2]:
#pip install llama-index
#!pip install transformers pdfplumber
!pip install ollama
#!pip install --upgrade pip

In [ ]:
from ollama import Client


client = Client(host='http://middleearthar.ddns.net:11434')


ModuleNotFoundError: No module named 'ollama'

In [ ]:
import requests

url = "https://api.wolframalpha.com/v1/result"
params = {
    "appid": "VLVTJ3-KW3TQPUGVG",  # Necesitas una clave API gratuita de Wolfram Alpha
    "i": "Eres un experto en geografía y tienes un conocimiento detallado de ciudades y países. Responde de manera detallada y en formato JSON: ¿Dónde queda Buenos Aires?"
}

response = requests.get(url, params=params)
if response.status_code == 200:
    print("Respuesta de Wolfram Alpha:", response.text)
else:
    print("Error:", response.status_code)


**LANGCHAIN OLLAMA MODEL:**

*PROBANDO AGREGADO DE HERRAMIENTAS COMO WIKIPEDIA Y DUCKDUCKGO

El código ahora también utiliza MariaDB como almacenamiento de memoria y LlamaIndex para una indexación rápida y búsqueda de consultas previas.

In [ ]:
import requests
import logging
import mariadb
import json
import webbrowser
import re
import nltk
from nltk.corpus import stopwords

# Descargar stopwords de NLTK si no están instaladas
# nltk.download('stopwords')

# Configurar logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Definición del modelo Ollama LLM (usando Ollama en Docker)
class OllamaLLM:
    def __init__(self, api_url="http://localhost:11434/api/generate", model="llama3.2:1b"):
        self.api_url = api_url
        self.model = model

    def generate(self, prompt: str, stream: bool = False):
        data = {
            "model": self.model,
            "prompt": prompt,
            "stream": stream
        }
        try:
            logging.debug(f"Enviando solicitud a Ollama con el modelo {self.model} y el prompt '{prompt}'")
            response = requests.post(self.api_url, json=data)
            response.raise_for_status()  # Lanzar excepción si hay error HTTP
            result = response.json().get('response', '')
            logging.debug(f"Respuesta de Ollama: {result}")
            
            if result.strip():  # Asegurarse de que la respuesta no esté vacía
                return result
            else:
                logging.error("Ollama devolvió una respuesta vacía.")
                return None
        except requests.Timeout:
            logging.error("Tiempo de espera agotado para Ollama.")
            return None
        except requests.RequestException as e:
            logging.error(f"Error en Ollama: {e}")
            return None

# Clase de memoria utilizando MariaDB
class MariaDBMemory:
    def __init__(self):
        try:
            self.connection = mariadb.connect(
                host='host.docker.internal',  # Cambiado para que coincida con el contenedor
                port=49154,  # Puerto especificado en la cadena de conexión
                user='root',
                password='mariadbpw',
                database='ollama_memory'  # Asegúrate de que esta base de datos exista
            )
            self.connection.autocommit = True
            print(f"SELFCONECTION: {self.connection}")
            logging.info("Conexión a MariaDB exitosa")
            self.create_table()  # Crear la tabla si no existe
        except mariadb.Error as err:
            logging.error(f"Error al conectar a MariaDB: {err}")
            self.connection = None

    def create_table(self):
        if self.connection is None:
            logging.error("No se puede crear la tabla: la conexión a MariaDB no fue establecida.")
            return
        cursor = self.connection.cursor()
        try:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS memory (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    prompt TEXT NOT NULL,
                    response LONGTEXT NOT NULL,  # Cambiado a LONGTEXT para respuestas largas
                    score INT DEFAULT 0
                );
            """)
            logging.info("Tabla 'memory' asegurada.")
        except mariadb.Error as err:
            logging.error(f"Error al crear la tabla: {err}")
        finally:
            cursor.close()

    def save_memory(self, prompt: str, response: dict, score: int):
        # Verificar que alguna de las respuestas no esté vacía
        valid_response = any(response.get(key) for key in ['ollama', 'wikipedia', 'duckduckgo'])

        if not valid_response:  # Si todas las respuestas son vacías o None
            logging.warning("No se guarda la respuesta vacía en la base de datos.")
            return

        if self.connection is None:
            logging.error("No se puede guardar en memoria: la conexión a MariaDB no fue establecida.")
            return
        cursor = self.connection.cursor()
        try:
            cursor.execute("""
                INSERT INTO memory (prompt, response, score)
                VALUES (%s, %s, %s);
            """, (prompt, str(response), score))  # Convertir el diccionario en cadena para almacenar
            logging.info(f"Memoria guardada: {prompt} con puntaje {score}")
        except mariadb.Error as err:
            logging.error(f"Error al guardar en memoria: {err}")
        finally:
            cursor.close()

    def retrieve_memory(self, prompt: str):
        if self.connection is None:
            logging.error("No se puede recuperar de memoria: la conexión a MariaDB no fue establecida.")
            return None
        cursor = self.connection.cursor()
        try:
            cursor.execute("""
                SELECT response, score FROM memory
                WHERE prompt = %s;
            """, (prompt,))
            result = cursor.fetchone()
            return result if result else None
        except mariadb.Error as err:
            logging.error(f"Error al recuperar de memoria: {err}")
            return None
        finally:
            cursor.close()

# Clase para obtener información de Wikipedia utilizando requests
class WikipediaTool:
    def __init__(self, language='es'):
        self.api_url = f"https://{language}.wikipedia.org/w/api.php"

    def get_summary(self, query: str):
        """Devuelve el resumen del artículo de Wikipedia utilizando requests."""
        params = {
            "action": "query",
            "prop": "extracts",
            "exintro": True,
            "titles": query,
            "format": "json"
        }
        try:
            logging.info(f"Haciendo solicitud a Wikipedia para: {query}")
            response = requests.get(self.api_url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                pages = data["query"]["pages"]
                for page_id, page_data in pages.items():
                    logging.info(f"Título encontrado: {page_data['title']}")
                    return page_data.get('extract', "No se encontró extracto disponible.")
            else:
                logging.error(f"Error al conectarse a Wikipedia: {response.status_code}")
                return None
        except requests.RequestException as e:
            logging.error(f"Error en la solicitud a Wikipedia: {e}")
            return None

    def get_full_article(self, query: str):
        """Devuelve el contenido completo del artículo de Wikipedia utilizando requests."""
        params = {
            "action": "query",
            "prop": "extracts",
            "titles": query,
            "format": "json"
        }
        try:
            logging.info(f"Haciendo solicitud a Wikipedia para: {query}")
            response = requests.get(self.api_url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                pages = data["query"]["pages"]
                for page_id, page_data in pages.items():
                    logging.info(f"Título encontrado: {page_data['title']}")
                    return page_data.get('extract', "No se encontró contenido disponible.")
            else:
                logging.error(f"Error al conectarse a Wikipedia: {response.status_code}")
                return None
        except requests.RequestException as e:
            logging.error(f"Error en la solicitud a Wikipedia: {e}")
            return None

# Clase para obtener información de DuckDuckGo
class DuckDuckGoTool:
    def __init__(self):
        self.api_url = "https://api.duckduckgo.com/"

    def get_answer(self, query: str):
        params = {
            'q': query,
            'format': 'json',
            "kl": "es-es"
        }
        try:
            response = requests.get(self.api_url, params=params)
            response.raise_for_status()  # Lanzar excepción si hay error
            data = response.json()
            if 'AbstractText' in data and data['AbstractText']:
                return data['AbstractText']
            else:
                return None  # Devuelve None si no encuentra nada
        except requests.RequestException as e:
            logging.error(f"Error en DuckDuckGo: {e}")
            return None

# Extender el modelo Ollama para que use la memoria en MariaDB
class OllamaWithMemory:
    def __init__(self, model: OllamaLLM, memory: MariaDBMemory):
        self.model = model
        self.memory = memory

    def generate_with_memory(self, prompt: str):
        # Intentar recuperar de la memoria
        memory_response = self.memory.retrieve_memory(prompt)
        if memory_response:
            logging.info("Respuesta encontrada en memoria.")
            return memory_response

        # Si no está en memoria, generar nueva respuesta
        logging.info("Generando nueva respuesta...")
        response = self.model.generate(prompt)
        if response:  # Solo guarda si hay una respuesta válida
            return response, 1  # Retornar respuesta con puntaje 1 si solo Ollama respondió
        return None, 0

# Integración de Ollama, Wikipedia y DuckDuckGo con memoria y sistema de puntaje
class OllamaWithMemoryAndTools(OllamaWithMemory):
    def __init__(self, model: OllamaLLM, memory: MariaDBMemory, tools: dict):
        super().__init__(model=model, memory=memory)
        self.tools = tools
        self.stop_words = set(stopwords.words('spanish'))

    def clean_question(self, question: str):
        """Limpia la pregunta eliminando stopwords y caracteres innecesarios."""
        # Convertir a minúsculas y eliminar caracteres especiales
        question_cleaned = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', question.lower())
        
        # Dividir en palabras y eliminar stopwords
        words = question_cleaned.split()
        important_words = [word for word in words if word not in self.stop_words]
        
        # Unir las palabras importantes de nuevo en una cadena
        return ' '.join(important_words)

    def ask(self, question: str, full_article=True):
        # Generar respuesta utilizando memoria y herramientas
        ollama_response, score = self.generate_with_memory(question)
        wikipedia_response = None
        duckduckgo_response = None

        # Limpiar la pregunta para extraer el tema
        cleaned_topic = self.clean_question(question)
        
        # Intentar recuperar de memoria primero
        memory_data = self.memory.retrieve_memory(cleaned_topic)
        if memory_data:
            logging.info(f"Datos encontrados en memoria para: {cleaned_topic}")
            responses, stored_score = json.loads(memory_data[0]), memory_data[1]
            
            # Verificamos si falta alguna respuesta (Wikipedia o DuckDuckGo)
            if not responses.get('wikipedia'):
                logging.info("No se encontró respuesta de Wikipedia, intentando recuperarla...")
                if full_article:
                    wikipedia_response = self.tools['wikipedia'].get_full_article(cleaned_topic)
                else:
                    wikipedia_response = self.tools['wikipedia'].get_summary(cleaned_topic)
                if wikipedia_response:
                    responses['wikipedia'] = wikipedia_response
                    score += 1
            
            if not responses.get('duckduckgo'):
                logging.info("No se encontró respuesta de DuckDuckGo, intentando recuperarla...")
                duckduckgo_response = self.tools['duckduckgo'].get_answer(cleaned_topic)
                if duckduckgo_response:
                    responses['duckduckgo'] = duckduckgo_response
                    score += 1

            # Si alguna respuesta faltaba y la recuperamos, actualizamos en memoria
            if wikipedia_response or duckduckgo_response:
                self.memory.save_memory(question, responses, score)

            return responses, score

        # Si no hay memoria, hacemos las consultas normalmente
        if cleaned_topic:
            logging.info(f"Buscando en Wikipedia: {cleaned_topic}")
            if full_article:
                wikipedia_response = self.tools['wikipedia'].get_full_article(cleaned_topic)  # Obtener artículo completo
            else:
                wikipedia_response = self.tools['wikipedia'].get_summary(cleaned_topic)  # Obtener solo el resumen
            
            if wikipedia_response:
                logging.info(f"Respuesta de Wikipedia recibida para {cleaned_topic}")
                logging.info(f"RESPUESTA WIKIPEDIA: {wikipedia_response}")
                score += 1
            else:
                logging.error(f"No se encontró información en Wikipedia para {cleaned_topic}")
        
        duckduckgo_response = self.tools['duckduckgo'].get_answer(cleaned_topic if cleaned_topic else question)
        if duckduckgo_response:
            logging.info(f"Respuesta de DuckDuckGo recibida para {cleaned_topic or question}")
            score += 1
        else:
            logging.error(f"No se encontró información en DuckDuckGo para {cleaned_topic or question}")

        # Guardar todas las respuestas separadas
        responses = {
            "ollama": ollama_response,
            "wikipedia": wikipedia_response,
            "duckduckgo": duckduckgo_response
        }

        if ollama_response:
            # Guardar solo si Ollama responde correctamente
            self.memory.save_memory(question, responses, score)

        return responses, score  # Retorna todas las respuestas por separado con puntaje




In [ ]:
# Ejemplo de uso
ollama_llm = OllamaLLM()
mariadb_memory = MariaDBMemory()

# Herramientas
wikipedia_tool = WikipediaTool(language="es")
duckduckgo_tool = DuckDuckGoTool()

tools = {
    "wikipedia": wikipedia_tool,
    "duckduckgo": duckduckgo_tool
}

# Crear la instancia de Ollama con memoria y herramientas
ollama_with_memory_and_tools = OllamaWithMemoryAndTools(
    model=ollama_llm, 
    memory=mariadb_memory, 
    tools=tools
)

# Hacer una pregunta
response = ollama_with_memory_and_tools.ask("¿Quién fue Albert Einstein?")
print("Respuesta:", response)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Selecciona el modelo que deseas usar
model_name = "EleutherAI/gpt-neo-2.7B"  # O "EleutherAI/gpt-j-6B"

# Cargar el modelo y el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Mover el modelo a GPU si está disponible
if torch.cuda.is_available():
    model = model.to("cuda")

# Función para generar texto
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Usar la función
prompt = "¿Cuál es el futuro de la inteligencia artificial?"
generated_text = generate_text(prompt)
print(generated_text)


In [ ]:
pip install stopwords

In [ ]:
pip install nltk